# Read data

In [1]:
import pandas as pd
import os

In [2]:
path = "data/MCQs.csv"
basename = os.path.basename(path)
basename = os.path.splitext(basename)[0]

mcq_df = pd.read_csv(path)
mcq_df.describe(include="all")

,question,options,correct_answer,correct_rationale,question_id
count,15,15,15,15,15.000000
unique,15,15,8,15,NaN
top,Which irrigation method is most efficient for ...,A. Drip Irrigation;B. Flood irrigation;C. Over...,B,"In dry regions, water is a limited resource an...",NaN
freq,1,1,4,1,NaN
mean,NaN,NaN,NaN,NaN,8.000000
std,NaN,NaN,NaN,NaN,4.472136
min,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,4.500000
50%,NaN,NaN,NaN,NaN,8.000000
75%,NaN,NaN,NaN,NaN,11.500000


In [3]:
mcq_df["correct_answer"].value_counts()

correct_answer
B          4
C          3
A          2
D          2
A, C, E    1
B, C, D    1
B, D       1
B, E       1
Name: count, dtype: int64

# Set up API keys

In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [5]:
def mcq_prompt(question, answer_options, country):
    # convert options to list
    answer_options = answer_options.replace(";", "\n")
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"For the following multiple choice question, please select the correct answer(s) from options A to E and "
            f"provide a detailed rationale for your choice. There can be multiple correct answers.\n\n"
            f"Question: {question}\n\n"
            f"Answer options:\n{answer_options}\n\n"
            f"Your response must follow this format exactly:\n\n"
            f"Correct answer: [OPTION(S)]\n\n"
            f"Rationale: [Not longer than 10 sentences]")

In [6]:
import litellm

# set callbacks
litellm.success_callback = ["langfuse"]
litellm.failure_callback = ["langfuse"]

[2025-07-07 01:34:28 - httpx:1025 - INFO] HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [7]:
country = "Kenya"

In [8]:
def get_llm_answers(df, model, country, **kwargs):
    ids = []
    prompts = []
    answers = []
    correct_answers = []
    correct_rationales = []

    for _, row in df.iterrows():
        question = row["question"]
        id = row["question_id"]
        answer_options = row["options"]
        correct_answer = row["correct_answer"]
        correct_rationale = row["correct_rationale"]
        prompt = mcq_prompt(question, answer_options, country)

        response = litellm.completion(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            **kwargs
        )

        # Extract the answer from the response
        answer = response["choices"][0]["message"]["content"]

        # Append the id, prompt, and answer to the lists
        ids.append(id)
        prompts.append(prompt)
        answers.append(answer)
        correct_answers.append(correct_answer)
        correct_rationales.append(correct_rationale)

    # Create a DataFrame from the lists
    results_df = pd.DataFrame({
        "question_id": ids,
        "prompt": prompts,
        "answer": answers,
        "correct_answer": correct_answers,
        "correct_rationale": correct_rationales
    })

    # Save the DataFrame to a CSV file
    model_name = model.replace("/", "-").replace(".", "-")
    csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
    print(f"Saving results to {csv_path}")
    results_df.to_csv(csv_path, index=False)

    return results_df

# OpenAI

In [9]:
gpt_4o_df = get_llm_answers(mcq_df, "openai/gpt-4o-2024-08-06", country)

Saving results to results/MCQs-openai-gpt-4o-2024-08-06.csv


In [10]:
gpt_4o_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A. Drip Irrigation\n\nRational...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: B, E\n\nRationale: Applying fe...",B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Intercropping ...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: The best pract...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: The best pract...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: The Ayrs...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: Boran and B...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Sorghum and...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: To contr...",C,Fall armyworm (FAW) is a destructive pest that...


In [11]:
o1_df = get_llm_answers(mcq_df, "openai/o1-2024-12-17", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/MCQs-openai-o1-2024-12-17.csv


In [12]:
o1_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: [A]\n\nRationale: Drip irrigat...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Randomly sprea...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Selecting mutu...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: [C]\n\nRationale: Applying a p...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Storing dry gr...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: [A, B, C, E]\n\nRationale: Ayr...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: Boran and B...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: [B, D]\n\nRationale: Sorghum a...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: [B]\n\nRationale: Contour farm...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: [B, C, D]\n\nRationale: Fungic...",C,Fall armyworm (FAW) is a destructive pest that...


In [13]:
o3_df = get_llm_answers(mcq_df, "openai/o3-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/MCQs-openai-o3-2025-04-16.csv


In [14]:
o3_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Effective fert...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Successful cer...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Effective weed...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Grain should f...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: C, E\n\nRationale: Friesians (...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: Jersey and ...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Sorghum and...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: Fall arm...",C,Fall armyworm (FAW) is a destructive pest that...


In [15]:
o4_mini_df = get_llm_answers(mcq_df, "openai/o4-mini-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/MCQs-openai-o4-mini-2025-04-16.csv


In [16]:
o4_mini_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: \n1. Drip irr...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Spot applicati...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Selecting comp...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Applying a pre...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: \n1. Hermetic...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: Ayrshire...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: \nBoran an...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Sorghum and...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: \nContour far...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: \n1. Re...",C,Fall armyworm (FAW) is a destructive pest that...


# Anthropic

In [17]:
claude_3_5_df = get_llm_answers(mcq_df, "anthropic/claude-3-5-sonnet-20241022", country)

Saving results to results/MCQs-anthropic-claude-3-5-sonnet-20241022.csv


In [18]:
claude_3_5_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: The most effec...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Successful leg...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Proper herbici...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Proper grain s...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: The Ayrs...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: The Sahi...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Sorghum, mi...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: Fall arm...",C,Fall armyworm (FAW) is a destructive pest that...


In [19]:
claude_3_7_df = get_llm_answers(mcq_df, "anthropic/claude-3-7-sonnet-20250219", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/MCQs-anthropic-claude-3-7-sonnet-20250219.csv


In [20]:
claude_3_7_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Proper fertili...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Option B repre...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C repre...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Option A combi...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: Ayrshire...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: Among the l...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Drought-pro...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: Fall arm...",C,Fall armyworm (FAW) is a destructive pest that...


Deepseek models don't support the `reasoning_effort` parameter.

In [21]:
deepseek_v3_df = get_llm_answers(mcq_df, "deepseek/deepseek-chat", country, max_completion_tokens=25000)

Saving results to results/MCQs-deepseek-deepseek-chat.csv


In [22]:
deepseek_v3_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A \n\nRationale: Drip irrigat...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: B, E \n\nRationale: \n- **Op...",B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: [B] \n\nRationale: Intercropp...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C \n\nRationale: Pre-emergenc...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: [A]\n\nRationale: Storing dry ...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: Ayrshire...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D \n\nRationale: Sahiwa...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D \n\nRationale: Sorghum a...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: \n- **B...",C,Fall armyworm (FAW) is a destructive pest that...


In [23]:
deepseek_r1_df = get_llm_answers(mcq_df, "deepseek/deepseek-reasoner", country, max_completion_tokens=25000)

Saving results to results/MCQs-deepseek-deepseek-reasoner.csv


In [24]:
deepseek_r1_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Option B is co...,B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: \nIntercroppin...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: Option C is co...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Storing dry ma...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, C, E\n\nRationale: Friesian...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: Boran (C) i...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Drought-pro...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: Contour farmin...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D\n\nRationale: \n- Opti...",C,Fall armyworm (FAW) is a destructive pest that...


# Gemini

In [25]:
gemini_2_5_pro_df = get_llm_answers(mcq_df, "gemini/gemini-2.5-pro-preview-05-06", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/MCQs-gemini-gemini-2-5-pro-preview-05-06.csv


In [26]:
gemini_2_5_pro_df

,question_id,prompt,answer,correct_answer,correct_rationale
0,1,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an..."
1,2,You provide smallholder farmers in Kenya with ...,"Correct answer: B, E\n\nRationale: The best wa...",B,Fertilizer application is a technique that req...
2,3,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale: This option pr...,B,Intercropping enhances productivity when compa...
3,4,You provide smallholder farmers in Kenya with ...,Correct answer: C\n\nRationale: The most effec...,C,Weed suppression before and after emergence of...
4,5,You provide smallholder farmers in Kenya with ...,Correct answer: A\n\nRationale: Storing proper...,A,Post harvest losses in grain occur due to pest...
5,6,You provide smallholder farmers in Kenya with ...,"Correct answer: A, B, C, E\n\nRationale: Fries...","A, C, E",Best livestock breeds are bred for high milk y...
6,7,You provide smallholder farmers in Kenya with ...,"Correct answer: C, D\n\nRationale: The Boran a...","B, C, D",Best livestock breeds for beef are bred for me...
7,8,You provide smallholder farmers in Kenya with ...,"Correct answer: B, D\n\nRationale: Sorghum and...","B, D",Best crops for drought prone areas are toleran...
8,9,You provide smallholder farmers in Kenya with ...,Correct answer: B\n\nRationale:\nContour farmi...,B,Contour farming is the most effective practice...
9,10,You provide smallholder farmers in Kenya with ...,"Correct answer: B, C, D, E\n\nRationale: An in...",C,Fall armyworm (FAW) is a destructive pest that...


# Aggregate results

In [27]:
basename

'MCQs'

In [28]:
import glob

# Get a list of all CSV files in the results directory
csv_files = glob.glob(f"results/{basename}-*.csv")
# Print the list of CSV files
print(csv_files)

['results/MCQs-openai-o3-2025-04-16.csv', 'results/MCQs-deepseek-deepseek-chat.csv', 'results/MCQs-anthropic-claude-3-7-sonnet-20250219.csv', 'results/MCQs-deepseek-deepseek-reasoner.csv', 'results/MCQs-anthropic-claude-3-5-sonnet-20241022.csv', 'results/MCQs-gemini-gemini-2-5-pro-preview-05-06.csv', 'results/MCQs-openai-o4-mini-2025-04-16.csv', 'results/MCQs-openai-o1-2024-12-17.csv', 'results/MCQs-openai-gpt-4o-2024-08-06.csv']


In [29]:
len(csv_files)

9

In [30]:
# read all csv files, extract the model name from the filename, and concatenate them into a single dataframe
dfs = []
for file in csv_files:
    model_name = "-".join(os.path.basename(file).split("-")[1:]).split(".")[0]
    df = pd.read_csv(file)
    df["model"] = model_name
    dfs.append(df)

# concatenate all dataframes into a single dataframe
all_df = pd.concat(dfs, ignore_index=True)

# drop the prompt column
all_df = all_df.drop(columns=["prompt"])

# add country column
all_df["country"] = country

In [31]:
all_df

,question_id,answer,correct_answer,correct_rationale,model,country
0,1,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an...",openai-o3-2025-04-16,Kenya
1,2,Correct answer: B\n\nRationale: Effective fert...,B,Fertilizer application is a technique that req...,openai-o3-2025-04-16,Kenya
2,3,Correct answer: B\n\nRationale: Successful cer...,B,Intercropping enhances productivity when compa...,openai-o3-2025-04-16,Kenya
3,4,Correct answer: C\n\nRationale: Effective weed...,C,Weed suppression before and after emergence of...,openai-o3-2025-04-16,Kenya
4,5,Correct answer: A\n\nRationale: Grain should f...,A,Post harvest losses in grain occur due to pest...,openai-o3-2025-04-16,Kenya
...,...,...,...,...,...,...
130,11,"Correct answer: D, E\n\nRationale: Banding aro...",D,The most effective fertilizer application meth...,openai-gpt-4o-2024-08-06,Kenya
131,12,"Correct answer: C, D\n\nRationale: Top-dressin...",C,"During top dressing, maize requires fertilizer...",openai-gpt-4o-2024-08-06,Kenya
132,13,"Correct answer: C, E\n\nRationale: During cold...","B, E","During cold weather, poultry is more vulnerabl...",openai-gpt-4o-2024-08-06,Kenya
133,14,Correct answer: D\n\nRationale: Crop rotation ...,D,Crop rotation does not lead to continuous use ...,openai-gpt-4o-2024-08-06,Kenya


In [32]:
# add the questions by merging with the original dataframe
all_df = all_df.merge(mcq_df[["question_id", "question"]], on="question_id", how="left")

In [33]:
all_df

,question_id,answer,correct_answer,correct_rationale,model,country,question
0,1,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an...",openai-o3-2025-04-16,Kenya,Which irrigation method is most efficient for ...
1,2,Correct answer: B\n\nRationale: Effective fert...,B,Fertilizer application is a technique that req...,openai-o3-2025-04-16,Kenya,How do I apply fertilizer in my farm to maximi...
2,3,Correct answer: B\n\nRationale: Successful cer...,B,Intercropping enhances productivity when compa...,openai-o3-2025-04-16,Kenya,What are the best practices in the intercroppi...
3,4,Correct answer: C\n\nRationale: Effective weed...,C,Weed suppression before and after emergence of...,openai-o3-2025-04-16,Kenya,Which is the best way to apply herbicide in my...
4,5,Correct answer: A\n\nRationale: Grain should f...,A,Post harvest losses in grain occur due to pest...,openai-o3-2025-04-16,Kenya,Which is the best practice for storing grain t...
...,...,...,...,...,...,...,...
130,11,"Correct answer: D, E\n\nRationale: Banding aro...",D,The most effective fertilizer application meth...,openai-gpt-4o-2024-08-06,Kenya,Which fertilizer application method is more ef...
131,12,"Correct answer: C, D\n\nRationale: Top-dressin...",C,"During top dressing, maize requires fertilizer...",openai-gpt-4o-2024-08-06,Kenya,Which fertilizer is suitable during top-dressi...
132,13,"Correct answer: C, E\n\nRationale: During cold...","B, E","During cold weather, poultry is more vulnerabl...",openai-gpt-4o-2024-08-06,Kenya,Which disease should I look at during cold wea...
133,14,Correct answer: D\n\nRationale: Crop rotation ...,D,Crop rotation does not lead to continuous use ...,openai-gpt-4o-2024-08-06,Kenya,Which one is not a benefit of crop rotation?


In [34]:
# reorder the columns
all_df = all_df[["model", "question_id", "question", "answer", "correct_answer", "correct_rationale", "country"]]
all_df

,model,question_id,question,answer,correct_answer,correct_rationale,country
0,openai-o3-2025-04-16,1,Which irrigation method is most efficient for ...,Correct answer: A\n\nRationale: Drip irrigatio...,A,"In dry regions, water is a limited resource an...",Kenya
1,openai-o3-2025-04-16,2,How do I apply fertilizer in my farm to maximi...,Correct answer: B\n\nRationale: Effective fert...,B,Fertilizer application is a technique that req...,Kenya
2,openai-o3-2025-04-16,3,What are the best practices in the intercroppi...,Correct answer: B\n\nRationale: Successful cer...,B,Intercropping enhances productivity when compa...,Kenya
3,openai-o3-2025-04-16,4,Which is the best way to apply herbicide in my...,Correct answer: C\n\nRationale: Effective weed...,C,Weed suppression before and after emergence of...,Kenya
4,openai-o3-2025-04-16,5,Which is the best practice for storing grain t...,Correct answer: A\n\nRationale: Grain should f...,A,Post harvest losses in grain occur due to pest...,Kenya
...,...,...,...,...,...,...,...
130,openai-gpt-4o-2024-08-06,11,Which fertilizer application method is more ef...,"Correct answer: D, E\n\nRationale: Banding aro...",D,The most effective fertilizer application meth...,Kenya
131,openai-gpt-4o-2024-08-06,12,Which fertilizer is suitable during top-dressi...,"Correct answer: C, D\n\nRationale: Top-dressin...",C,"During top dressing, maize requires fertilizer...",Kenya
132,openai-gpt-4o-2024-08-06,13,Which disease should I look at during cold wea...,"Correct answer: C, E\n\nRationale: During cold...","B, E","During cold weather, poultry is more vulnerabl...",Kenya
133,openai-gpt-4o-2024-08-06,14,Which one is not a benefit of crop rotation?,Correct answer: D\n\nRationale: Crop rotation ...,D,Crop rotation does not lead to continuous use ...,Kenya


In [35]:
# save the dataframe to a CSV file
all_df.to_csv(f"{basename}-all.csv", index=False)